In [20]:
from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D
from keras.layers import Activation, Dense, Dropout, Flatten, InputLayer
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard
from keras.callbacks import ModelCheckpoint,EarlyStopping
from keras.models import Sequential, load_model
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray
from skimage.io import imsave
import numpy as np
import tensorflow as tf
import os

from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()

In [2]:
# mount your drive once per colab session
from google.colab import drive
drive.mount('/content/drive')
# !unzip '/content/drive/My Drive/train'.zip -d '/content/drive/My Drive/' # only need to do once for train and test

Mounted at /content/drive


In [ ]:
# unzip the test files!
#!unzip '/content/drive/My Drive/test'.zip -d '/content/drive/My Drive/' # only need this once

In [3]:
# factor needed for cifar10
factor = int(256/32)
model = Sequential()
model.add(InputLayer(input_shape=(int(256 / factor), int(256 / factor), 1)))
model.add(Conv2D(64/factor, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(64/factor, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(128/factor, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128/factor, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(256/factor, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256/factor, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(512/factor, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256/factor, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128/factor, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64/factor, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32/factor, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))
model.add(UpSampling2D((2, 2)))
model.compile(optimizer='rmsprop', loss='mse')

In [13]:
# Get images, colab
X = []
i = 0
for filename in os.listdir('/content/drive/My Drive/train/'):
  X.append(img_to_array(load_img('/content/drive/My Drive/train/' + filename)))
  # more data takes sooo much time, something around 13000
  if i == 10000:
    break
  i += 1
    
X = np.array(X, dtype=float)

# Set up train and test data
Xtrain = X[:len(X) - 1]
Xtrain = 1.0/255*Xtrain


In [24]:
# Set up train and test data
split = int(0.95*len(X))
Xtrain = X[:split]

#splitting validation
Xtrain = 1.0/255*Xtrain
split_idx = int(0.9*len(Xtrain))
Xvalid=Xtrain[split_idx:]
Xtrain=Xtrain[:split_idx]

# Image transformer
datagen = ImageDataGenerator(
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=20,
        horizontal_flip=True)

# Image transformer
batch_size = 100
def train_generator(batch_size):
    for batch in datagen.flow(Xtrain, batch_size=batch_size):
        lab_batch = rgb2lab(batch)
        X_batch = lab_batch[:,:,:,0]
        Y_batch = lab_batch[:,:,:,1:] / 128
        yield (X_batch.reshape(X_batch.shape+(1,)), Y_batch)

def validationGenerator(batch_size):
    for batch in datagen.flow(Xvalid, batch_size=batch_size):
        lab_batch = rgb2lab(batch)
        X_batch = lab_batch[:,:,:,0]
        Y_batch = lab_batch[:,:,:,1:] / 128
        yield (X_batch.reshape(X_batch.shape+(1,)), Y_batch)

# new stuff from Naveen
steps_per_epoch = int(len(Xtrain)/batch_size)
validation_steps = int(len(Xvalid)/batch_size)

In [25]:
# Train model   

filepath = "/content/drive/My Drive/Model/model.h5"

#model.load_weights(filepath)

checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
callbacks_list = [es,checkpoint]

tensorboard = TensorBoard(log_dir="output/first_run")
model.fit(train_generator(batch_size), callbacks=callbacks_list, epochs=1000, steps_per_epoch=steps_per_epoch,
          validation_data=validationGenerator(batch_size), validation_steps=validation_steps)

Epoch 1/1000
85/85 [==============================] - ETA: 0s - batch: 42.0000 - size: 100.0000 - loss: 0.0111

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '



Epoch 00001: loss improved from inf to 0.01106, saving model to /content/drive/My Drive/Model/model.h5
85/85 [==============================] - 45s 75ms/step - batch: 42.0000 - size: 100.0000 - loss: 0.0111 - val_loss: 0.0103
Epoch 2/1000
85/85 [==============================] - ETA: 0s - batch: 42.0000 - size: 99.4118 - loss: 0.0108
Epoch 00002: loss improved from 0.01106 to 0.01077, saving model to /content/drive/My Drive/Model/model.h5
85/85 [==============================] - 6s 68ms/step - batch: 42.0000 - size: 99.4118 - loss: 0.0108 - val_loss: 0.0104
Epoch 3/1000
85/85 [==============================] - ETA: 0s - batch: 42.0000 - size: 99.4118 - loss: 0.0106
Epoch 00003: loss improved from 0.01077 to 0.01059, saving model to /content/drive/My Drive/Model/model.h5
85/85 [==============================] - 6s 71ms/step - batch: 42.0000 - size: 99.4118 - loss: 0.0106 - val_loss: 0.0100
Epoch 4/1000
85/85 [==============================] - ETA: 0s - batch: 42.0000 - size: 99.4118 - 

In [26]:
# Save model
model_json = model.to_json()
with open("/content/drive/My Drive/Model/model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("/content/drive/My Drive/Model/model.h5")

In [62]:
from skimage import color

# Test images for cifar-10
test_folder_cifar = '/content/drive/My Drive/test/'
files_sorted = os.listdir(test_folder_cifar)
files_sorted = [x for x in files_sorted if x.endswith(".png")]
files_sorted.sort(key=lambda x:int(x.split(".")[0]))

color_me = []
#should be after your training data
for i in range(1,500):
    filename = files_sorted[i]
    fname = test_folder_cifar + filename
    color_me.append(img_to_array(load_img(fname)))
color_me = np.array(color_me, dtype=float)
color_me = rgb2lab(1.0 / 255 * color_me)[:, :, :, 0]
color_me = color_me.reshape(color_me.shape + (1,))

print("Testing model")

# Test model
output = model.predict(color_me)
output = output * 128

# Output colorizations
for i in range(len(output)):
    cur = np.zeros((int(256 / factor), int(256 / factor), 3))
    cur[:, :, 0] = color_me[i][:, :, 0]
    cur[:, :, 1:] = output[i]
    imsave("/content/drive/My Drive/Results/" + str(i+1) + ".png", lab2rgb(cur))

Testing model


In [61]:
color_me = []
test_folder_cifar = '/content/drive/My Drive/'

#should be after your training data
fname = test_folder_cifar + "cat.png"
color_me.append(img_to_array(load_img(fname)))
color_me = np.array(color_me, dtype=float)
color_me = rgb2lab(1.0 / 255 * color_me)[:, :, :, 0]
color_me = color_me.reshape(color_me.shape + (1,))

print("Testing model")

# Test model
output = model.predict(color_me)
output = output * 128

# Output colorizations
cur = np.zeros((int(256 / factor), int(256 / factor), 3))
cur[:, :, 0] = color_me[0][:, :, 0]
cur[:, :, 1:] = output[0]
imsave("/content/drive/My Drive/Results/" + str(i+1) + ".png", lab2rgb(cur))

Testing model
